# Homework Week 07
Nicola Zomer <br>
19 December, 2022

# Assignment
Calculate the avalanche duration probability $P_>(t)$ if $\lambda i = \lambda$ for all $t$ and all neurons. This leads, as only sketched in class, to the following integral: 
$$
    \gamma \int_0^\infty d\lambda \cdot \exp(-\gamma\lambda)\cdot(1-\exp(-\delta\lambda))^n
$$
which can be solved through the saddle point approximation.

Optional. Create a time series $\lambda(t)$, with $t=1,2,...,T$ where at each time $t$, the value of $\lambda$ is extracted from an exponential distribution. Then simulate $N=100$ independent heterogeneous Poisson processes, where each one describes the spikes events of a single neuron, but all have the same time dependent rate parameter $\lambda(t)$.

## Calculation of the avalanche duration probability
The derivation of the above integral and its solution using the Saddle Point approximation is given in the attached pdf.

## Simulation